# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Format today and yesterday into strings for filename creation
format = "%m-%d-%Y"
now = datetime.now()
today = now.strftime(format)
yesterday = (now - timedelta(days=1)).strftime(format)

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
print(jh_live_date)

12-13-2022


In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-12-14 04:21:04,33.93911,67.709953,206743,7843,NaN,NaN,Afghanistan,531.086079,3.793599
1,NaN,NaN,NaN,Albania,2022-12-14 04:21:04,41.15330,20.168300,333567,3594,NaN,NaN,Albania,11591.041768,1.077445
2,NaN,NaN,NaN,Algeria,2022-12-14 04:21:04,28.03390,1.659600,271149,6881,NaN,NaN,Algeria,618.341051,2.537719
3,NaN,NaN,NaN,Andorra,2022-12-14 04:21:04,42.50630,1.521800,47446,158,NaN,NaN,Andorra,61406.846567,0.333010
4,NaN,NaN,NaN,Angola,2022-12-14 04:21:04,-11.20270,17.873900,104808,1925,NaN,NaN,Angola,318.892306,1.836692


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/4/22,12/5/22,12/6/22,12/7/22,12/8/22,12/9/22,12/10/22,12/11/22,12/12/22,12/13/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,206206,206273,206331,206414,206465,206504,206543,206603,206702,206743
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,333408,333413,333455,333472,333490,333491,333521,333533,333535,333567
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271107,271113,271122,271128,271135,271140,271146,271146,271147,271149


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/4/22,12/5/22,12/6/22,12/7/22,12/8/22,12/9/22,12/10/22,12/11/22,12/12/22,12/13/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7834,7835,7835,7837,7839,7839,7839,7839,7840,7843
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3594,3594,3594,3594,3594,3594,3594,3594,3594,3594
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6881,6881,6881,6881,6881,6881,6881,6881,6881,6881


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/4/22,12/5/22,12/6/22,12/7/22,12/8/22,12/9/22,12/10/22,12/11/22,12/12/22,12/13/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/4/22,12/5/22,12/6/22,12/7/22,12/8/22,12/9/22,12/10/22,12/11/22,12/12/22,12/13/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,18680,18680,18680,18752,18752,18752,18752,18752,18752,18752
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,66730,66730,66730,66951,66951,66951,66951,66951,66951,66951
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,6980,6980,6980,6989,6989,6989,6989,6989,6989,6989


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/4/22,12/5/22,12/6/22,12/7/22,12/8/22,12/9/22,12/10/22,12/11/22,12/12/22,12/13/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,230,230,230,230,230,230,230,230,230,230
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,716,716,716,717,717,717,717,717,717,717
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,103,103,103,103,103,103,103,103,103,103


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
678,1001.0,Autauga,Alabama,US,2022-12-14 04:21:04,32.539527,-86.644082,18752,230,NaN,NaN,"Autauga, Alabama, US",33564.230611,1.226536
715,1075.0,Lamar,Alabama,US,2022-12-14 04:21:04,33.779950,-88.096680,4537,70,NaN,NaN,"Lamar, Alabama, US",32864.904020,1.542870
716,1077.0,Lauderdale,Alabama,US,2022-12-14 04:21:04,34.901719,-87.656247,28906,414,NaN,NaN,"Lauderdale, Alabama, US",31172.556590,1.432229


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,99526290,1085224,0.0
India,44675095,530658,0.0
France,38815319,161049,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,99526290,1085224,0.0,98441066.0,2022-12-14 04:21:04,37.934269,-91.444083
India,44675095,530658,0.0,44144437.0,2022-12-14 04:21:04,23.088275,81.806127
France,38815319,161049,0.0,38654270.0,2022-12-14 04:21:04,6.430808,-34.730285
Germany,36859058,159367,0.0,36699691.0,2022-12-14 04:21:04,51.571844,10.277881
Brazil,35696918,691178,0.0,35005740.0,2022-12-14 04:21:04,-12.669522,-48.480493


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,11633186,97892,0.0
Texas,8137427,91516,0.0
Florida,7270410,83201,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,11633186,97892,0.0,11535294.0,2022-12-14 04:21:04,37.843962,-120.728594
Texas,8137427,91516,0.0,8045911.0,2022-12-14 04:21:04,31.660643,-98.653069
Florida,7270410,83201,0.0,7187209.0,2022-12-14 04:21:04,28.940755,-82.700744
New York,6464457,74414,0.0,6390043.0,2022-12-14 04:21:04,42.544151,-75.474183
Illinois,3918785,40441,0.0,3878344.0,2022-12-14 04:21:04,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,3587829,34352,0.0
Florida,Miami-Dade,1482343,11920,0.0
Arizona,Maricopa,1470301,18167,0.0
Illinois,Cook,1467530,14989,0.0
Texas,Harris,1230094,11389,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,3587829,34352,0.0,3553477.0,2022-12-14 04:21:04,34.308284,-118.228241,6037.0
Florida,Miami-Dade,1482343,11920,0.0,1470423.0,2022-12-14 04:21:04,25.611236,-80.551706,12086.0
Arizona,Maricopa,1470301,18167,0.0,1452134.0,2022-12-14 04:21:04,33.348359,-112.491815,4013.0
Illinois,Cook,1467530,14989,0.0,1452541.0,2022-12-14 04:21:04,41.841448,-87.816588,17031.0
Texas,Harris,1230094,11389,0.0,1218705.0,2022-12-14 04:21:04,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-11,206603,333533,271146,47446,104808,11,9106,9766975,445828,10862500,...,995402,247838,11981,548348,11520639,703036,535,11945,333746,259356
2022-12-12,206702,333535,271147,47446,104808,11,9106,9766975,445881,10866876,...,998047,248210,11981,548455,11529639,703228,535,11945,333746,259356
2022-12-13,206743,333567,271149,47446,104808,11,9106,9766975,445881,10870758,...,998047,248268,11981,548570,11521388,703228,535,11945,333746,259356


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-11,7839,3594,6881,158,1925,0,146,130041,8712,16462,...,7541,1637,14,5830,43178,5708,0,2159,4019,5622
2022-12-12,7840,3594,6881,158,1925,0,146,130041,8712,16482,...,7548,1637,14,5830,43178,5708,0,2159,4019,5622
2022-12-13,7843,3594,6881,158,1925,0,146,130041,8712,16492,...,7548,1637,14,5830,43179,5708,0,2159,4019,5622


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-12-12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-12-13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/4/22,12/5/22,12/6/22,12/7/22,12/8/22,12/9/22,12/10/22,12/11/22,12/12/22,12/13/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,18680,18680,18680,18752,18752,18752,18752,18752,18752,18752
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,4524,4524,4524,4537,4537,4537,4537,4537,4537,4537
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,28777,28777,28777,28906,28906,28906,28906,28906,28906,28906
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,9015,9015,9015,9037,9037,9037,9037,9037,9037,9037
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,44730,44730,44730,44889,44889,44889,44889,44889,44889,44889


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-12-11,1555092,300365,8265,2353530,974616,11623042,1718077,933631,318567,49,...,2399246,8135739,1066450,147221,23633,2163370,1866343,617094,1937599,181695
2022-12-12,1555092,300365,8265,2353530,974793,11630825,1720013,934388,318968,49,...,2399246,8136509,1066450,147221,23655,2167396,1866343,617812,1940474,181695
2022-12-13,1555092,300719,8265,2353530,975637,11633186,1720938,935123,319063,49,...,2399246,8137427,1066450,147221,23667,2169118,1866343,618099,1941830,182198


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-12-11       18752   66951    6989  7653  17559    2855   6127   39125   
2022-12-12       18752   66951    6989  7653  17559    2855   6127   39125   
2022-12-13       18752   66951    6989  7653  17559    2855   6127   39125   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-12-11        10204     6330  ...    7580   2235     9647     2282   
2022-12-12        10204     6330  ...    7580   2235     9647     2282   
2022-12-13        10204     6330  ...    7589   2240     9689     2288   

Province_State                                                     
Admin2         Sweetwater  Teton Uinta Unassigned Washakie Weston  
2022-12-11          12349  11947  6272          0     2701   1862  
2022-12-12          12349  11947  6272          0     2701   1862  
2022-12-13          12374  11968  6287          0     2717   1871  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-12-11,20682,1436,34,31822,12631,97834,13704,11603,3184,0,...,28406,91514,5138,781,126,22538,14829,7638,15686,1938
2022-12-12,20682,1436,34,31822,12631,97873,13715,11603,3184,0,...,28406,91516,5138,781,126,22548,14829,7646,15689,1938
2022-12-13,20682,1436,34,31822,12638,97892,13740,11656,3184,0,...,28406,91516,5138,781,126,22559,14829,7649,15697,1950


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-12-11         230     717     103  108    260      54    130     661   
2022-12-12         230     717     103  108    260      54    130     661   
2022-12-13         230     717     103  108    260      54    130     661   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-12-11          170       87  ...     151     43       84       28   
2022-12-12          170       87  ...     151     43       84       28   
2022-12-13          170       87  ...     151     44       86       28   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-12-11            134    16    43          0       47     22  
2022-12-12            134    16    43          0       47     22  
2022-12-13            135    16    43          0       47     22  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change().round(7)
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-11,0.000291,0.000036,0.000000,0.0,0.0,0.0,0.0,0.002784,0.000000,0.000265,...,0.000000,0.000000,0.0,0.000361,0.000017,0.000000,0.0,0.0,0.0,0.0
2022-12-12,0.000479,0.000006,0.000004,0.0,0.0,0.0,0.0,0.000000,0.000119,0.000403,...,0.002657,0.001501,0.0,0.000195,0.000781,0.000273,0.0,0.0,0.0,0.0
2022-12-13,0.000198,0.000096,0.000007,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000357,...,0.000000,0.000234,0.0,0.000210,-0.000716,0.000000,0.0,0.0,0.0,0.0


In [60]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [61]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change().round(7)
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-11,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,0.000054,0.0,0.000000,...,0.000000,0.0,0.0,0.000172,0.000000,0.0,NaN,0.0,0.0,0.0
2022-12-12,0.000128,0.0,0.0,0.0,0.0,NaN,0.0,0.000000,0.0,0.001215,...,0.000928,0.0,0.0,0.000000,0.000000,0.0,NaN,0.0,0.0,0.0
2022-12-13,0.000383,0.0,0.0,0.0,0.0,NaN,0.0,0.000000,0.0,0.000607,...,0.000000,0.0,0.0,0.000000,0.000023,0.0,NaN,0.0,0.0,0.0


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [63]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change().round(7)
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [65]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change().round(7)
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-12-11,0.0,0.000000,0.0,0.0,0.000237,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2022-12-12,0.0,0.000000,0.0,0.0,0.000182,0.000670,0.001127,0.000811,0.001259,0.0,...,0.0,0.000095,0.0,0.0,0.000931,0.001861,0.0,0.001164,0.001484,0.000000
2022-12-13,0.0,0.001179,0.0,0.0,0.000866,0.000203,0.000538,0.000787,0.000298,0.0,...,0.0,0.000113,0.0,0.0,0.000507,0.000794,0.0,0.000465,0.000699,0.002768


In [66]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [67]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change().round(7)
jh_hist_uscounties_cases_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-12-11         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-12-12         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-12-13         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ...   Wyoming                                \
Admin2         Chambers Cherokee  ...      Park    Platte  Sheridan  Sublette   
2022-12-11          0.0      0.0  ...  0.000000  0.000000  0.000000  0.000000   
2022-12-12          0.0      0.0  ...  0.000000  0.000000  0.000000  0.000000   
2022-12-13          0.0      0.0  ...  0.001187  0.002237  0.004354  0.002629   

Province_State                                                                
Admin2         Sweetwater     Teton     Uinta Unassigned  Washakie    Weston  
2022-12-11       0.000000  0.000000  0.000000        NaN  0.000000  0.000000  
2022-12-12       0.000000  0.000000  0.000000        NaN  0.000000  0.000000  
2022-12-13       0.002024  0.001758  0.002392        NaN  0.005924  0.004834  

[3 rows x 3336 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change().round(7)
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-12-11,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,NaN,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
2022-12-12,0.0,0.0,0.0,0.0,0.000000,0.000399,0.000803,0.000000,0.0,NaN,...,0.0,0.000022,0.0,0.0,0.0,0.000444,0.0,0.001047,0.000191,0.000000
2022-12-13,0.0,0.0,0.0,0.0,0.000554,0.000194,0.001823,0.004568,0.0,NaN,...,0.0,0.000000,0.0,0.0,0.0,0.000488,0.0,0.000392,0.000510,0.006192


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change().round(7)
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-12-11         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-12-12         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-12-13         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                              \
Admin2         Chambers Cherokee  ...    Park    Platte Sheridan Sublette   
2022-12-11          0.0      0.0  ...     0.0  0.000000  0.00000      0.0   
2022-12-12          0.0      0.0  ...     0.0  0.000000  0.00000      0.0   
2022-12-13          0.0      0.0  ...     0.0  0.023256  0.02381      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-12-11       0.000000   0.0   0.0        NaN      0.0    0.0  
2022-12-12       0.000000   0.0   0.0        NaN      0.0    0.0  
2022-12-13       0.007463   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [72]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [73]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean().round(7)
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-11,0.000253,0.000048,0.000011,0.000152,0.000023,0.0,0.0,0.001397,0.000002,0.000776,...,0.000012,0.000139,0.0,0.000219,0.000028,0.000000,0.0,0.0,0.0,0.000013
2022-12-12,0.000366,0.000027,0.000007,0.000076,0.000012,0.0,0.0,0.000699,0.000060,0.000589,...,0.001335,0.000820,0.0,0.000207,0.000405,0.000137,0.0,0.0,0.0,0.000006
2022-12-13,0.000282,0.000062,0.000007,0.000038,0.000006,0.0,0.0,0.000349,0.000030,0.000473,...,0.000667,0.000527,0.0,0.000208,-0.000156,0.000068,0.0,0.0,0.0,0.000003


In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [75]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-11,0.000025,0.0,0.0,0.000201,0.000004,NaN,0.0,0.000027,1.800000e-06,0.000768,...,0.000004,0.0,0.0,0.000086,4.000000e-07,0.0,NaN,0.0,0.0,0.000006
2022-12-12,0.000076,0.0,0.0,0.000100,0.000002,NaN,0.0,0.000014,9.000000e-07,0.000991,...,0.000466,0.0,0.0,0.000043,2.000000e-07,0.0,NaN,0.0,0.0,0.000003
2022-12-13,0.000229,0.0,0.0,0.000050,0.000001,NaN,0.0,0.000007,5.000000e-07,0.000799,...,0.000233,0.0,0.0,0.000022,1.170000e-05,0.0,NaN,0.0,0.0,0.000002


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [77]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean().round(7)
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-12-11,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-12-12,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-12-13,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [78]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [79]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-12-11,0.000118,0.000026,0.000017,0.000215,0.000421,0.000237,0.000160,0.000413,0.000159,0.0,...,1.800000e-06,0.000144,0.000378,0.000095,0.000124,0.000211,0.000110,0.000121,0.000181,0.000067
2022-12-12,0.000059,0.000013,0.000008,0.000107,0.000301,0.000453,0.000643,0.000612,0.000709,0.0,...,9.000000e-07,0.000119,0.000189,0.000048,0.000528,0.001036,0.000055,0.000642,0.000833,0.000033
2022-12-13,0.000029,0.000596,0.000004,0.000054,0.000584,0.000328,0.000591,0.000699,0.000503,0.0,...,5.000000e-07,0.000116,0.000095,0.000024,0.000517,0.000915,0.000028,0.000553,0.000766,0.001401


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [81]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-12-11      0.000121  0.000104  0.000041  0.000066  0.000106  0.000022   
2022-12-12      0.000061  0.000052  0.000020  0.000033  0.000053  0.000011   
2022-12-13      0.000030  0.000026  0.000010  0.000016  0.000027  0.000006   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-12-11      0.000129  0.000100  0.000152  0.000344  ...  0.000033   
2022-12-12      0.000064  0.000050  0.000076  0.000172  ...  0.000017   
2022-12-13      0.000032  0.000025  0.000038  0.000086  ...  0.000602   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-12-11      0.000071  0.000102  0.000062   0.000058  0.000048  0.000043   
2022-12-12      0.000035  0.000051  0.000031   0.000029  0.000024  0.000021   
2022-12-13      0.001136  0.002202  0.001330   0.001027  0.000891  0.001207   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-12-11      -0.503937  0.000117  0.000017  
2022-12-12      -0.503937  0.000058  0.000009  
2022-12-13      -0.503937  0.002991  0.002421  

[3 rows x 3336 columns]

In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [83]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-12-11,0.000046,0.0,0.0,0.000070,0.000343,0.000121,0.000195,0.000023,0.000473,NaN,...,2.000000e-06,0.000036,0.000271,0.000446,0.000252,0.000181,0.000262,0.000063,0.000130,4.000000e-07
2022-12-12,0.000023,0.0,0.0,0.000035,0.000171,0.000260,0.000499,0.000012,0.000236,NaN,...,1.000000e-06,0.000029,0.000136,0.000223,0.000126,0.000312,0.000131,0.000555,0.000160,2.000000e-07
2022-12-13,0.000011,0.0,0.0,0.000018,0.000363,0.000227,0.001161,0.002290,0.000118,NaN,...,5.000000e-07,0.000015,0.000068,0.000112,0.000063,0.000400,0.000066,0.000474,0.000335,3.096100e-03


In [84]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [85]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State Alabama                                                  \
Admin2         Autauga   Baldwin Barbour Bibb    Blount Bullock Butler   
2022-12-11         0.0  0.000044     0.0  0.0  0.000122     0.0    0.0   
2022-12-12         0.0  0.000022     0.0  0.0  0.000061     0.0    0.0   
2022-12-13         0.0  0.000011     0.0  0.0  0.000030     0.0    0.0   

Province_State                             ...       Wyoming            \
Admin2         Calhoun  Chambers Cherokee  ...          Park    Platte   
2022-12-11         0.0  0.000185      0.0  ...  8.000000e-07  0.000000   
2022-12-12         0.0  0.000092      0.0  ...  4.000000e-07  0.000000   
2022-12-13         0.0  0.000046      0.0  ...  2.000000e-07  0.011628   

Province_State                                                                \
Admin2          Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2022-12-11      0.000000      0.0   0.000003   0.0   0.0       -1.0      0.0   
2022-12-12      0.000000      0.0   0.000001   0.0   0.0       -1.0      0.0   
2022-12-13      0.011905      0.0   0.003732   0.0   0.0       -1.0      0.0   

Province_State         
Admin2         Weston  
2022-12-11        0.0  
2022-12-12        0.0  
2022-12-13        0.0  

[3 rows x 3336 columns]

In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [87]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

HTTPError: HTTP Error 403: Forbidden

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [ ]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [ ]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

In [ ]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

#### 